In [1]:
#include "src/tilemap.hpp"
#include "src/player.hpp"
#include "src/enemylist.hpp"

In [ ]:
SVG canvas = SVG(1500, 1600, "CyberCrawler 2077");

TileMap tileMap = TileMap(&canvas);

EnemyList enemyList = EnemyList(&tileMap, &canvas);
Player player = Player(4, 5, &tileMap, &enemyList, &canvas);

std::string key = canvas.lastKey();
while(key != "p") {
    key = canvas.lastKey();
    player.keyMovement(key, &enemyList);
    for(int i = 0; i < 4; i++) {
        enemyList.getEnemy(i) -> autoMove();
    }
}